In [ ]:
!pip install pillow

import numpy as np
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
from google.colab import files
import io

ASCII_CHARS_DETAILED = ["@", "#", "S", "%", "?", "*", "+", ";", ":", ",", "."]
ASCII_CHARS_SIMPLE = ["@", "%", "#", "*", "+", "=", "-", ":", ".", " "]
ASCII_CHARS_BLOCKS = ["█", "▓", "▒", "░", " "]
ASCII_CHARS_MINIMAL = ["#", "+", ".", " "]

def resize_image(image, new_width=100):
    width, height = image.size
    aspect_ratio = height / width
    new_height = int(aspect_ratio * new_width * 0.55)
    resized_image = image.resize((new_width, new_height))
    return resized_image

def grayscale_image(image):
    return image.convert("L")

def pixels_to_ascii(image, ascii_chars=ASCII_CHARS_SIMPLE):
    pixels = image.getdata()
    ascii_str = ""
    range_width = 256 // len(ascii_chars)
    for pixel in pixels:
        char_index = min(pixel // range_width, len(ascii_chars) - 1)
        ascii_str += ascii_chars[char_index]
    return ascii_str

def ascii_to_image(ascii_str, width, font_size=10):
    if ascii_str is None or width is None:
        return None
    ascii_lines = [ascii_str[i:i+width] for i in range(0, len(ascii_str), width)]
    char_width = font_size * 0.6
    char_height = font_size
    img_width = int(width * char_width)
    img_height = int(len(ascii_lines) * char_height)
    image = Image.new('RGB', (img_width, img_height), color='black')
    draw = ImageDraw.Draw(image)
    try:
        font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSansMono.ttf", font_size)
    except:
        font = ImageFont.load_default()
    y_offset = 0
    for line in ascii_lines:
        draw.text((0, y_offset), line, fill='white', font=font)
        y_offset += char_height
    return image

def image_to_ascii(image_path, new_width=100, ascii_chars=ASCII_CHARS_SIMPLE):
    try:
        image = Image.open(image_path)
        image = resize_image(image, new_width)
        image = grayscale_image(image)
        ascii_str = pixels_to_ascii(image, ascii_chars)
        img_width = image.width
        ascii_lines = [ascii_str[i:i+img_width] for i in range(0, len(ascii_str), img_width)]
        ascii_art = "\n".join(ascii_lines)
        return ascii_art, image.width
    except Exception as e:
        print(f"Error: {e}")
        return None, None

print("Upload your image:")
uploaded = files.upload()

if uploaded:
    filename = list(uploaded.keys())[0]

    original_img = Image.open(filename)
    plt.figure(figsize=(8, 8))
    plt.imshow(original_img)
    plt.axis('off')
    plt.title('Original Image')
    plt.show()

    print("Generating ASCII art...")

    ascii_art_detailed, width_detailed = image_to_ascii(filename, 100, ASCII_CHARS_DETAILED)
    ascii_art_simple, width_simple = image_to_ascii(filename, 100, ASCII_CHARS_SIMPLE)
    ascii_art_blocks, width_blocks = image_to_ascii(filename, 80, ASCII_CHARS_BLOCKS)
    ascii_art_minimal, width_minimal = image_to_ascii(filename, 100, ASCII_CHARS_MINIMAL)

    if ascii_art_simple and width_simple:
        fig, axes = plt.subplots(2, 2, figsize=(16, 16))
        fig.suptitle('ASCII Art Gallery')

        img1 = ascii_to_image(ascii_art_detailed, width_detailed, 8)
        if img1:
            axes[0, 0].imshow(img1)
            axes[0, 0].set_title('DETAILED')
            axes[0, 0].axis('off')

        img2 = ascii_to_image(ascii_art_simple, width_simple, 8)
        if img2:
            axes[0, 1].imshow(img2)
            axes[0, 1].set_title('SIMPLE')
            axes[0, 1].axis('off')

        img3 = ascii_to_image(ascii_art_blocks, width_blocks, 10)
        if img3:
            axes[1, 0].imshow(img3)
            axes[1, 0].set_title('BLOCKS')
            axes[1, 0].axis('off')

        img4 = ascii_to_image(ascii_art_minimal, width_minimal, 8)
        if img4:
            axes[1, 1].imshow(img4)
            axes[1, 1].set_title('MINIMAL')
            axes[1, 1].axis('off')

        plt.tight_layout()
        plt.show()

        print("\nText version:")
        print(ascii_art_simple)

        output_filename = filename.rsplit('.', 1)[0] + '_ascii.txt'
        with open(output_filename, 'w') as f:
            f.write(ascii_art_simple)

        ascii_img = ascii_to_image(ascii_art_simple, width_simple, 10)
        img_filename = filename.rsplit('.', 1)[0] + '_ascii.png'
        ascii_img.save(img_filename)

        files.download(output_filename)
        files.download(img_filename)

        print("Done!")
    else:
        print("Failed to generate ASCII art")
else:
    print("No file uploaded")